In [1]:
#Importing libraries to be used
import numpy as np
import random
import sys
import io
import os
import glob
import IPython
from td_utils import *
import time
import tensorflow as tf
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
Tx = 5511 # The number of time steps input to the model from the spectrogram
n_freq = 101 # Number of frequencies input to the model at each time step of the spectrogram
Ty = 1375 # The number of time steps in the output of our model

In [3]:
# Load preprocessed training examples
X = np.load("./XY_Train/XTrain_Final3.npy")
Y = np.load("./XY_Train/YTrain_Final3.npy")

In [4]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

Using TensorFlow backend.


In [5]:
#Enabling Automatic Mixed Precision for Training
import os
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

In [6]:
#Defining the model
def model(input_shape):
    """
    Function creating the model's graph in Keras.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """
    
    X_input = Input(shape = input_shape)
    
       
    # Step 1: CONV layer (≈4 lines)
    X = Conv1D(196, kernel_size=15, strides=4)(X_input)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)

    # Step 2: First GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X) # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)
    X = BatchNormalization()(X)                                 # Batch normalization
    
    # Step 3: Second GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X)   # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)
    X = BatchNormalization()(X)                                  # Batch normalization
    X = Dropout(0.8)(X)                                  # dropout (use 0.8)
    
    # Step 4: Time-distributed dense layer (≈1 line)
    X = TimeDistributed(Dense(1, activation = "sigmoid"))(X) # time distributed  (sigmoid)


    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [8]:
#Using Otimisation Algorithms for Gradient Descent and compiling the model
model = model(input_shape = (Tx, n_freq))
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [9]:
#Training the model and finding out the time it takes to run 'n' epochs
start=time.time()
model.fit(X, Y, batch_size =64, epochs=10)
print(time.time()-start)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
70/70 [==============================] - 14s 201ms/step - loss: 0.6970 - acc: 0.9261
Epoch 2/10
70/70 [==============================] - 9s 136ms/step - loss: 0.7014 - acc: 0.5073
Epoch 3/10
70/70 [==============================] - 9s 135ms/step - loss: 0.7133 - acc: 0.5070
Epoch 4/10
70/70 [==============================] - 9s 135ms/step - loss: 0.7016 - acc: 0.5125
Epoch 5/10
70/70 [==============================] - 9s 134ms/step - loss: 0.6943 - acc: 0.5237
Epoch 6/10
70/70 [==============================] - 9s 134ms/step - loss: 0.6981 - acc: 0.5204
Epoch 7/10
70/70 [==============================] - 9s 134ms/step - loss: 0.6956 - acc: 0.5289
Epoch 8/10
70/70 [==============================] - 9s 133ms/step - loss: 0.6906 - acc: 0.5410
Epoch 9/10
70/70 [==============================] - 9s 134ms/step - loss: 0.6900 - acc: 0.5449
Epoch 10/10
70/70 [==============================] - 9s 133ms/step - loss: 0.6885 - acc: 0.5484
